In [1]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

In [2]:
import os
import subprocess
import json

temp_dir="outputs/temp1/"
if not os.path.exists(temp_dir):
    os.mkdir(temp_dir)

model_name = "meta-llama/Llama-2-7b-chat-hf"
test_dataset_path="../smallDB/1018ig/qa.json"
context_path = "../smallDB/1018ig/context_ig_paraphrase_plus_oa.json"
ds_config_file_model="ds/zero_infer.json"
ds_config_file_train="ds/zero_train.json"

temp_model_dir="outputs/temp_model"
log_filepath="results/1025ds/1.json"


In [3]:
for epochs in [1,2,3,5]:
    for n_contexts in [1,2,5,12,62,112,212,512,1012,2012,5012]:
        run_id=f"{epochs}-{n_contexts}"
        with open(context_path) as f:
            context = json.load(f)

        sel_context = context[:n_contexts]

        sel_context_path=temp_dir+f"{run_id}context.json"
        with open(sel_context_path, 'w') as f:
            json.dump(sel_context, f)


        #train
        subprocess.run([
            'python', 'src/DStrain.py',
            '--model_name', model_name,
            '--context_path', sel_context_path,
            '--ds_config_file_model', ds_config_file_model,
            '--ds_config_file_train', ds_config_file_train,
            '--temp_model_dir', temp_model_dir,
            '--log_filepath', log_filepath,
            '--epochs', str(epochs),
            '--run_id', str(run_id)
        ])

        #eval
        subprocess.run([
            'python', 'src/DSeval.py',
            '--temp_model_dir', temp_model_dir,
            '--test_dataset_path', test_dataset_path,
            '--log_filepath', log_filepath,
            '--run_id', (run_id)
        ])

[2023-10-25 01:18:34,257] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/user/miniconda3/envs/ft/lib/python3.11/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


[2023-10-25 01:18:35,655] [INFO] [comm.py:637:init_distributed] cdb=None
[2023-10-25 01:18:35,655] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2023-10-25 01:18:35,703] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=192.168.11.2, master_port=29500
[2023-10-25 01:18:35,703] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2023-10-25 01:18:47,880] [INFO] [partition_parameters.py:347:__exit__] finished initializing model - num_params = 291, num_elems = 6.74B


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.35s/it]


[2023-10-25 01:18:51,217] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.10.3, git-hash=unknown, git-branch=unknown
[2023-10-25 01:18:51,230] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2023-10-25 01:18:51,231] [INFO] [logging.py:96:log_dist] [Rank 0] Creating ZeRO Offload
[2023-10-25 01:18:51,309] [INFO] [utils.py:803:see_memory_usage] DeepSpeedZeRoOffload initialize [begin]
[2023-10-25 01:18:51,310] [INFO] [utils.py:804:see_memory_usage] MA 0.06 GB         Max_MA 0.86 GB         CA 1.04 GB         Max_CA 1 GB 
[2023-10-25 01:18:51,310] [INFO] [utils.py:811:see_memory_usage] CPU Virtual Memory:  used = 24.23 GB, percent = 9.6%
Parameter Offload: Total persistent parameters: 266240 in 65 params
[2023-10-25 01:18:51,399] [INFO] [utils.py:803:see_memory_usage] DeepSpeedZeRoOffload initialize [end]
[2023-10-25 01:18:51,400] [INFO] [utils.py:804:see_memory_usage] MA 0.06 GB         Max_MA 0.06 GB         CA 1.04 GB         Max_CA 1 

...

Using /home/user/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Emitting ninja build file /home/user/.cache/torch_extensions/py311_cu117/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module cpu_adam...


Parameter Offload: Total persistent parameters: 266240 in 65 params
[2023-10-25 01:19:24,163] [WARNING] [lr_schedules.py:751:__init__] total_num_steps 1 is less than warmup_num_steps 0


  0%|          | 0/1 [00:00<?, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
100%|██████████| 1/1 [00:16<00:00, 16.89s/it]


{'train_runtime': 16.8916, 'train_samples_per_second': 0.059, 'train_steps_per_second': 0.059, 'train_loss': 2.442516565322876, 'epoch': 1.0}


KeyboardInterrupt: 